# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
2. Công cụ sử dụng
3. Các bước thu thập dữ liệu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import những  thư viện cần thiết
import pandas as pd
import numpy as np
!pip install requests_html
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import math
import requests
import time

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.5 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=60f86c9d37ff86f1575b0f7651886c59869c7ef7a62fb0c86fc6f45e3505d7a3
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [4]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        print("Success!!!")
        return response
    return None

In [5]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [6]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()
for work_method in range(1,7):
   #https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6
   url = BASE_URL + '/tim-kiem-viec-lam-nhanh?&working_method={}'.format(work_method)
   print(url)
   page_content = get_content(url, headers)
   soup = BeautifulSoup(page_content.text, 'html.parser')

   buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
   buffer2 = buffer1.find('span',class_="font-semibold").text
   n_jobs = int(buffer2.replace(',',''))
   # Trung bình mỗi page có 30 công việc
   n_pages = min(51, math.ceil(n_jobs/30) + 1)
   print('Có {} công việc, {} trang ở work_method = {}'.format(n_jobs, n_pages, work_method))


   for page in range(1,n_pages):
      # Khởi tạo url vầ yêu cầu lấy page về
      url = BASE_URL + '/tim-kiem-viec-lam-nhanh?page={}&working_method={}'.format(page, work_method)
      print(url)
      page_content = get_content(url, headers)
      soup = BeautifulSoup(page_content.text, 'html.parser')

      containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
      print('Đã tìm thấy {} công việc'.format(len(containers)))
      for container in containers:
         dict_job = {}

         buffer = BASE_URL + container.find('a', href=True)['href']
         dict_job['Liên kết'] = buffer if buffer else None

         buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
         dict_job['Tên công việc'] = buffer.text if buffer else None

         buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
         dict_job['Tên công ty'] = buffer.text if buffer else None

         buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
         dict_job['Mức lương'] = buffer.text if buffer else None

         buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
         dict_job['Khu vực tuyển'] = buffer.text if buffer else None

         job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)



https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=1
Success!!!
Có 5655 công việc, 51 trang ở work_method = 1
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=3&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=4&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=5&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=6&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=7&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=8&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
564,https://vieclam24h.vn/dien-dien-tu-dien-lanh/n...,Nhân Viên Kỹ Thuật Điện - Đi Làm Ngay,Công Ty Cổ Phần Sg Hoàng Minh,15 - 20 triệu,TP.HCM
1881,https://vieclam24h.vn/ngan-hang/thuc-tap-sinh-...,Thực Tập Sinh Tài Trợ Thương Mại - Hà Nội / Hồ...,Ngân Hàng TMCP Việt Nam Thịnh Vượng - Vpbank,0.1 - 3 triệu,"Hà Nội, TP.HCM"
1034,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Nhân Viên Tư Vấn Tín Dụng (Kênh Trả Góp) Tại T...,Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Thừa Thiên Huế
591,https://vieclam24h.vn/marketing/nhan-vien-digi...,Nhân Viên Digital Marketing,Công Ty TNHH Bánh Kẹo Tân Thế Kỷ,13 - 15 triệu,TP.HCM
781,https://vieclam24h.vn/khoa-hoc-ky-thuat/nhan-v...,Nhân Viên Kỹ Thuật Vận Hành Máy Sản Xuất Tại C...,CÔNG TY TNHH SẢN XUẤT THƯƠNG MẠI THY LONG,10 - 12 triệu,TP.HCM


In [7]:
# Lưu dữ liệu vào file csv
job_pool.to_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv', encoding='utf-8-sig', index=False)
# job_pool.to_csv('../data/job_pool.csv', encoding='utf-8-sig', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [11]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv', encoding='utf-8-sig')
# df = pd.read_csv('../data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
611,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Nhân Viên Tư Vấn Tín Dụng - Kênh Trả Góp Tại G...,Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Gia Lai
111,https://vieclam24h.vn/y-te-cham-soc-suc-khoe/d...,Điều Dưỡng Viên,CÔNG TY TNHH MỘT THÀNH VIÊN VĂCXIN VÀ SINH PHẨ...,15 - 20 triệu,Hà Nội
1567,https://vieclam24h.vn/dien-dien-tu-dien-lanh/n...,"Nhân Viên Kỹ Thuật Máy Tính, Camera",Công Ty Cổ Phần Thương Mại Dịch Vụ Vùng Trời T...,7 - 8 triệu,TP.HCM
1724,https://vieclam24h.vn/van-tai-lai-xe-giao-nhan...,Tài Xế Lái Xe Nâng (Thời Vụ),Công Ty TNHH Xe Nâng Miền Nam,8 - 12 triệu,TP.HCM
1825,https://vieclam24h.vn/thuc-tap-sinh/thuc-tap-s...,Thực Tập Sinh Kinh Doanh,Công Ty Cổ Phần Thủy Canh Miền Nam,1 - 5 triệu,TP.HCM


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       1916 non-null   object
 1   Tên công việc  1916 non-null   object
 2   Tên công ty    1916 non-null   object
 3   Mức lương      1916 non-null   object
 4   Khu vực tuyển  1916 non-null   object
dtypes: object(5)
memory usage: 75.0+ KB


In [15]:
raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])
for i, url in enumerate(df['Liên kết']):
    print("Công việc thứ {}".format(i))
    job_content = get_content(url, headers=headers)
    if (job_content != None):
      soup = BeautifulSoup(job_content.text, 'html.parser')

    dict_job = {}

    dict_job['Liên kết'] = url

    overview = soup.find('div', class_='md:ml-7 w-full')

    buffer = overview.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = overview.find_all('a', class_ ="hover:text-se-accent")
    dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None


    buffer = soup.find('p', string="Cấp bậc")
    dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Số lượng tuyển")
    dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Hình thức làm việc")
    dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu bằng cấp")
    dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
    dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Ngành nghề")
    dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


    buffer1 = soup.find('h4', string="Mô tả công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Yêu cầu công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Quyền lợi")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Địa điểm làm việc")
    buffer2 = buffer1.find_next_sibling('ul')
    buffer3 = buffer2.find_all('li', class_ = "jsx-d84db6a84feb175e mb-2 flex text-14")
    dict_job['Địa điểm làm việc'] = '; '.join([element.text for element in buffer3 if element.text.strip()]) if buffer3 else None


    buffer1 = soup.find('h3', string="Từ khoá")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Từ khóa'] = buffer2.text if buffer2 else None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    for buffer in buffers:
        if buffer.get_text() == 'Địa chỉ:':
            dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
        if buffer.get_text() == 'Quy mô:':
            dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

    raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

Công việc thứ 0
Success!!!
Công việc thứ 1
Success!!!
Công việc thứ 2
Success!!!
Công việc thứ 3
Success!!!
Công việc thứ 4
Success!!!
Công việc thứ 5
Success!!!
Công việc thứ 6
Success!!!
Công việc thứ 7
Success!!!
Công việc thứ 8
Success!!!
Công việc thứ 9
Success!!!
Công việc thứ 10
Success!!!
Công việc thứ 11
Success!!!
Công việc thứ 12
Success!!!
Công việc thứ 13
Success!!!
Công việc thứ 14
Success!!!
Công việc thứ 15
Success!!!
Công việc thứ 16
Success!!!
Công việc thứ 17
Success!!!
Công việc thứ 18
Success!!!
Công việc thứ 19
Success!!!
Công việc thứ 20
Success!!!
Công việc thứ 21
Success!!!
Công việc thứ 22
Success!!!
Công việc thứ 23
Success!!!
Công việc thứ 24
Success!!!
Công việc thứ 25
Success!!!
Công việc thứ 26
Success!!!
Công việc thứ 27
Success!!!
Công việc thứ 28
Success!!!
Công việc thứ 29
Success!!!
Công việc thứ 30
Success!!!
Công việc thứ 31
Success!!!
Công việc thứ 32
Success!!!
Công việc thứ 33
Success!!!
Công việc thứ 34
Success!!!
Công việc thứ 35
Success!!!
Cô

In [18]:
raw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
488,https://vieclam24h.vn/hanh-chinh-thu-ky/nhan-v...,Công Ty Cổ Phần Ffc,Nhân Viên Hành Chính Nhân Sự,9 - 15 triệu,Hà Nội,Chuyên viên- nhân viên,1,Toàn thời gian cố định,Đại học,3 năm,Hành chính - Thư ký/Nhân sự,"Hà Nội, Số nhà 18, Đường Louis 7, Khu đô thị L...",Về công tác Văn thư lưu trữ- Thực hiện quản lý...,- Tốt nghiệp Đại học chuyên ngành: Quản trị nh...,- Lương thỏa thuận (NET) Từ 9.000.000đ - 15.00...,huyện từ liêmkhu công nghiệp từ liêmbắc từ liê...,"Số nhà 18, Đường Louis 7, Khu đô thị Louis Cit...",10 - 150 người
1456,https://vieclam24h.vn/thiet-ke-sang-tao-nghe-t...,Công Ty Cổ Phần Dịch Vụ Guru,Nhân Viên Thiết Kế 2D & 3D,8 - 17 triệu,TP.HCM,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Cao đẳng,2 năm,Thiết kế - Sáng tạo nghệ thuật/Thông tin - Tru...,"TP.HCM, 12/12 Hoàng Bật Đạt, Phường 15, Tân Bình","- Thiết kế 2D: key visual, posm, những ấn phẩm...",- Có kiến thức và kỹ năng trong việc thiết ...,- Lương: Thương lượng (từ 8- 17triệu) tuỳ theo...,2dnhân viên thiết kế 2dthiết kế 2dViệc làm TP....,"18/22 Tăng Bạt Hổ, Phường 11, Quận Bình Thạnh,...",10 - 150 người
330,https://vieclam24h.vn/ban-hang-kinh-doanh/mien...,Home Credit Việt Nam,[Miền Bắc] Nhân Viên Thu Hồi Tín Dụng Thị Trường,12 - 20 triệu,"Quảng Ninh, Hưng Yên, Lạng Sơn, Bắc Giang, Nam...",Chuyên viên- nhân viên,10,Toàn thời gian cố định,Trung học,Chưa có kinh nghiệm,Bán hàng - Kinh doanh/Tài chính - Đầu tư - Chứ...,"Quảng Ninh, Uông Bí, Quảng Yên, Đông Triều, Uô...","Đầu tháng, nhận danh sách khách hàng đã sử dụn...","Nam/Nữ, độ tuổi từ > 32Tốt nghiệp từ trung học...",Tổng thu nhập từ 12 triệu /tháng trở lênLương ...,tín dụngthị trườngnhân viên thu hồi tín dụngVi...,"20 Nguyễn Đăng Giai, P. Thảo Điền, Quận 2, TP.HCM",Trên 300 người
777,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Công Ty Cổ Phần Pizza 4Ps,[HCM] Phó Quản Lý Bếp (Kitchen Assistant Manag...,12 - 15 triệu,TP.HCM,Quản lý nhóm- giám sát,1,Toàn thời gian cố định,None,1 năm,Khách sạn - Nhà hàng - Du lịch,"TP.HCM, 47 Tôn Thất Thiệp, Phường Bến Nghé, Qu...",WHAT WILL YOU DO? Làm việc tại: Nhà hàng Mì Ra...,"ARE YOUR ""FIT""? - Có kinh nghiệm 1-2 năm quản ...",WHAT IS YOUR BENEFITS?- Lương cơ bản 12.000.00...,hcm quận 1bếpViệc làm TP.HCMKhách sạn - Nhà hà...,"47 Tôn Thất Thiệp, Phường Bến Nghe, Quận 1",150 - 300 người
363,https://vieclam24h.vn/ban-si-ban-le-quan-ly-cu...,Công Ty Cổ Phần Tập Đoàn Vàng Bạc Đá Quý Doji,Cửa Hàng Trưởng Trung Tâm Trang Sức Tại Lào Cai,15 - 25 triệu,Lào Cai,Quản lý nhóm- giám sát,1,Toàn thời gian cố định,Đại học,3 năm,Bán sỉ - Bán lẻ - Quản lý cửa hàng/Dệt may - D...,"Lào Cai, 124 Đ. Hoàng Liên, Cốc Lếu, Lào Cai",- Quản lý các công việc chung của chi nhánh DO...,"- Tốt nghiệp Đại học hệ chính quy trở lên, chu...","- Được hưởng chế độ BHYT, BHXH, BHTN theo quy ...",cửa hàngcửatrungViệc làm Lào CaiBán sỉ - Bán l...,Hội sở: Tầng 7 - Tòa nhà DOJI Tower - Số 5 Lê ...,Trên 300 người


In [19]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
# raw_job_details.to_csv('../data/raw_dataset.csv', encoding='utf-8-sig', index=False)
raw_job_details.to_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/raw_dataset.csv', encoding='utf-8-sig', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             1916 non-null   object
 1   Tên công ty          1916 non-null   object
 2   Tên công việc        1916 non-null   object
 3   Mức lương            1916 non-null   object
 4   Khu vực tuyển        1916 non-null   object
 5   Cấp bậc              1916 non-null   object
 6   Số lượng tuyển       1916 non-null   object
 7   Hình thức làm việc   1916 non-null   object
 8   Yêu cầu bằng cấp     1676 non-null   object
 9   Yêu cầu kinh nghiệm  1916 non-null   object
 10  Ngành nghề           1916 non-null   object
 11  Địa điểm làm việc    1916 non-null   object
 12  Mô tả công việc      1916 non-null   object
 13  Yêu cầu công việc    1916 non-null   object
 14  Quyền lợi            1916 non-null   object
 15  Từ khóa              1916 non-null   object
 16  Địa ch

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [ ]:
# Demo lất page content
url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6'
page_content = get_content(url, headers)
page_text = page_content.text
soup = BeautifulSoup(page_text, 'html.parser')
buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
buffer2 = buffer1.find('span',class_="font-semibold").text
n_jobs = int(buffer2.replace(',',''))
print('Có {} công việc'.format(n_jobs))
n_pages = min(51, math.ceil(n_jobs/30) + 1)
print('Có {} trang'.format(n_pages-1))

Success!!!
Có 132 công việc
Có 5 trang


In [ ]:
# Demo lất page content

url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6'
page_content = get_content(url, headers)
page_text = page_content.text

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}

    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None

    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None

    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Success!!!
Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/ban-hang-kinh-doanh/thuc...,Thực Tập Kinh Doanh Dịch Vụ Mạng Viễn Thông Fpt,Công Ty Cổ Phần Viễn Thông Fpt,8 - 9 triệu,Hà Nội
1,https://vieclam24h.vn/hanh-chinh-thu-ky/thuc-t...,Thực Tập Sinh Hỗ Trợ Kinh Doanh,Công Ty TNHH Thương Mại Và Dịch Vụ Y Việt,1 - 2 triệu,TP.HCM
2,https://vieclam24h.vn/ban-si-ban-le-quan-ly-cu...,Thực Tập Sinh Kinh Doanh,Công Ty TNHH TM DV MTV Thực Phẩm Quốc Tế Vũ Yến,1 - 2 triệu,TP.HCM
3,https://vieclam24h.vn/y-te-cham-soc-suc-khoe/t...,Thực Tập Sinh Kỹ Sư,Công Ty TNHH Thương Mại Và Dịch Vụ Y Việt,2 - 4 triệu,TP.HCM
4,https://vieclam24h.vn/y-te-cham-soc-suc-khoe/t...,Thực Tập Sinh Product Admin,Công Ty TNHH Thương Mại Và Dịch Vụ Y Việt,3 - 4 triệu,TP.HCM
5,https://vieclam24h.vn/san-xuat-lap-rap-che-bie...,Thực Tập Sinh Điện Công Nghiệp / Điện Lạnh,Công Ty TNHH SX-TM-DV Nam Cường,5 - 8 triệu,Bình Dương
6,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Thực Tập Sinh Phục Vụ - F&b,Khách Sạn Win Suites,0.1 - 0.2 triệu,TP.HCM
7,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Thực Tập Sinh Buồng Phòng (House Keeping),Khách Sạn Win Suites,0.1 - 0.2 triệu,TP.HCM
8,https://vieclam24h.vn/ban-hang-kinh-doanh/thuc...,Thực Tập Sinh Kinh Doanh Thiết Bị Ngành Kỹ Thuật,Công Ty TNHH Thương Mại Dịch Vụ Ist,2 - 8 triệu,TP.HCM
9,https://vieclam24h.vn/nghe-nghiep-khac/thuc-ta...,Thực Tập Sinh Hành Chính - Nhân Sự,Công Ty TNHH Aic Việt Nam,2 - 3 triệu,Hà Nội


In [ ]:

url = 'https://vieclam24h.vn/tai-chinh-dau-tu-chung-khoan/chuyen-vien-dieu-tra-an-ninh-phong-chong-gian-lan-c18p122id200289988.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

Success!!!


In [ ]:
# Demo lấy chi tiết

raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

dict_job = {}

dict_job['Liên kết'] = url

overview = soup.find('div', class_='md:ml-7 w-full')

buffer = overview.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
dict_job['Tên công ty'] = buffer.text if buffer else None

buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
dict_job['Tên công việc'] = buffer.text if buffer else None

buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
dict_job['Mức lương'] = buffer.text if buffer else None

buffer = overview.find_all('a', class_ ="hover:text-se-accent")
dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None


buffer = soup.find('p', string="Cấp bậc")
dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', string="Số lượng tuyển")
dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', string="Hình thức làm việc")
dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', string="Yêu cầu bằng cấp")
dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', string="Ngành nghề")
dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


buffer1 = soup.find('h4', string="Mô tả công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', string="Yêu cầu công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', string="Quyền lợi")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', string="Địa điểm làm việc")
buffer2 = buffer1.find_next_sibling('ul')
buffer3 = buffer2.find_all('li', class_ = "jsx-d84db6a84feb175e mb-2 flex text-14")
print(buffer3[0].text)
dict_job['Địa điểm làm việc'] = '; '.join([element.text for element in buffer3 if element.text.strip()]) if buffer3 else None


buffer1 = soup.find('h3', string="Từ khoá")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Từ khóa'] = buffer2.text if buffer2 else None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
for buffer in buffers:
    if buffer.get_text() == 'Địa chỉ:':
        dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
    if buffer.get_text() == 'Quy mô:':
        dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
raw_job_details


TP.HCM, Tòa Nhà Phụ Nữ, 20 Nguyễn Đăng Giai, Thủ Đức


,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
0,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Home Credit Việt Nam,Chuyên Viên Điều Tra An Ninh - Phòng Chống Gia...,14 - 16 triệu,"TP.HCM, Đồng Nai, Bình Định, Bình Thuận",Chuyên viên- nhân viên,5,Toàn thời gian cố định,Trung cấp,1 năm,Tài chính - Đầu tư - Chứng Khoán/Ngân hàng/Luậ...,"TP.HCM, Tòa Nhà Phụ Nữ, 20 Nguyễn Đăng Giai, T...",*Theo Nghị định số 13/2023/NĐ-CP v/v bảo vệ dữ...,Tốt nghiệp Trung cấp trở lên (Ưu tiên tốt nghi...,Mức lương cạnh tranh (Đóng BHXH trên 100% lươn...,an ninhchuyên viênan ninhViệc làm TP.HCMTài c...,"20 Nguyễn Đăng Giai, P. Thảo Điền, Quận 2, TP.HCM",Trên 300 người
